In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np
import optuna
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import holidays 

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression

import catboost as cb
from scalecast.Forecaster import Forecaster

import lightgbm as lgb
import torch

import time

alt.data_transformers.disable_max_rows()
sklearn.set_config(transform_output="pandas")

In [183]:

holidays.US(years=2025)

{datetime.date(2025, 1, 1): "New Year's Day", datetime.date(2025, 5, 26): 'Memorial Day', datetime.date(2025, 6, 19): 'Juneteenth National Independence Day', datetime.date(2025, 7, 4): 'Independence Day', datetime.date(2025, 9, 1): 'Labor Day', datetime.date(2025, 11, 11): 'Veterans Day', datetime.date(2025, 11, 27): 'Thanksgiving', datetime.date(2025, 12, 25): 'Christmas Day', datetime.date(2025, 1, 20): 'Martin Luther King Jr. Day', datetime.date(2025, 2, 17): "Washington's Birthday", datetime.date(2025, 10, 13): 'Columbus Day'}

In [186]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


## Import Data

In [39]:
train = pd.read_csv('train.csv')
blind = pd.read_csv('test.csv')

In [6]:
#data_profile = ProfileReport(train)

In [42]:

def is_holiday(row):
    """
    Determines if a given date is a holiday in the specified country.
    Optimized for Canada, Finland, and Italy.
    
    Parameters:
    row: pandas Series containing 'country' and 'date' columns
    
    Returns:
    bool: True if the date is a holiday in the given country, False otherwise
    """
    country = row['country']
    date = pd.to_datetime(row['date']).date()  # Convert to date only

    country_mapping = {
        'Canada': holidays.CA(),
        'Finland': holidays.FI(),
        'Italy': holidays.IT(), 
        'Kenya': holidays.KE(),
        'Norway': holidays.NO(),
        'Singapore': holidays.SG(),
    }
    
    try:
        # Get the holidays object for the country
        if country in country_mapping:
            country_holidays = country_mapping[country]
            return date in country_holidays
        else:
            return False
    except Exception as e:
        print(f"Error checking holiday for {country} on {date}: {e}")
        return False

# Create a cache of holiday objects to improve performance
def initialize_holiday_detection(df):
    """
    Initialize holiday detection by creating holiday objects for all unique years in the dataset.
    This improves performance by avoiding repeated creation of holiday objects.
    
    Parameters:
    df: pandas DataFrame containing 'date' column
    """
    years = pd.to_datetime(df['date']).dt.year.unique()
    holiday_cache = {
        'Canada': {year: holidays.CA(years=year) for year in years},
        'Finland': {year: holidays.FI(years=year) for year in years},
        'Italy': {year: holidays.IT(years=year) for year in years}, 
        'Kenya': {year: holidays.KE(years=year) for year in years},
        'Norway': {year: holidays.NO(years=year) for year in years},
        'Singapore': {year: holidays.SG(years=year) for year in years}

    }
    
    def is_holiday_cached(row):
        country = row['country']
        date = pd.to_datetime(row['date']).date()
        year = date.year
        
        try:
            if country in holiday_cache:
                return date in holiday_cache[country][year]
            return False
        except Exception as e:
            print(f"Error checking holiday for {country} on {date}: {e}")
            return False
    
    return is_holiday_cached

# Example usage:
# First initialize the cached version
holiday_checker = initialize_holiday_detection(train)

# Then apply it to create the holiday flag
train['is_holiday'] = train.apply(holiday_checker, axis=1)

In [47]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    


    # Drop missing values and irrelevant columns
    imputer = SimpleImputer(strategy="median")  # Change to "mean", "most_frequent", or "constant"
    
    df = df.set_index('date')
    df = df.drop(columns=['id'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    try: 
        df["num_sold"] = imputer.fit_transform(df[["num_sold"]])
    except: pass
    try: 
        X = df.drop(columns=['num_sold'])
        X = pd.get_dummies(X)

    except:
        X=pd.get_dummies(df)
    try:
        y = df['num_sold']
    except:
        y = None
    # Scale features
    #X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (195610, 19)
y_train shape: (195610,)
X_test shape: (34520, 19)
y_test shape: (34520,)


In [48]:
model = LinearRegression()

# Train the model using training data
model.fit(X_train.values, y_train.values)

# Make predictions on the test data
y_pred = model.predict(X_test.values)

# Evaluate the model using common metrics
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Mean Absolute Error: {mae:.4f}")
print(f"MAPE: {mape:.4f}")

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

# Optionally: You can get the coefficients and intercept of the linear regression model
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Mean Absolute Error: 239.6190
MAPE: 3.2049
Mean Squared Error: 109541.0906
R-squared: 0.6742
Coefficients: [ 3.61205001e+01  6.86410539e-02  2.38087569e+01 -2.99911700e+00
 -6.35812851e+00  6.81236178e+01 -4.00209836e+00 -2.08989529e+02
 -6.63598094e+02  7.13789478e+02  9.46766248e+01 -3.12797164e+02
  2.25828312e+02  8.69688512e+01 -4.79094090e+02  4.96165201e+02
  2.75448520e+02 -1.93938551e+02 -9.85810797e+01]
Intercept: 13489.12723115343


In [25]:
def objective(trial):
    print(f"🔄 Starting Trial {trial.number + 1} out of {5}")  # Replace 20 with your actual n_trials

    # Define hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),
        "enable_categorical": True,  # Enable categorical features
        "device": "gpu",  # Ensure GPU usage
        "tree_method": "hist"  # Recommended for GPU acceleration in XGBoost 2.0+
    }

    # Train the model
    model = xgb.XGBRegressor(**param)
    model.fit(X_train.values, y_train.values)

    # Predict & compute MAPE
    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)
    #blind_preds = model.predict(blind)
    #blind_mape = mean_absolute_percentage_error(y_train, blind_preds)
    print(f"✅ Completed Trial {trial.number + 1} - MAPE: {mape:.4f}")

    return mape

# Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)  # ✅ Use explicit n_trials here


[I 2025-01-30 12:28:57,527] A new study created in memory with name: no-name-f7ba9455-e99d-46a3-99c2-7388e9a638fd


🔄 Starting Trial 1 out of 5


c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\xgboost\core.py:158: UserWarning: [12:28:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2025-01-30 12:28:58,865] Trial 0 finished with value: 0.6605830691622739 and parameters: {'n_estimators': 271, 'learning_rate': 0.18793297710581836, 'max_depth': 5, 'subsample': 0.9718692862623313, 'colsample_bytree': 0.5352327519748985, 'reg_alpha': 1.934644828288758, 'reg_lambda': 1.2046216208166824}. Be

✅ Completed Trial 1 - MAPE: 0.6606
🔄 Starting Trial 2 out of 5


[I 2025-01-30 12:29:19,601] Trial 1 finished with value: 0.477071541779303 and parameters: {'n_estimators': 479, 'learning_rate': 0.10395377307741967, 'max_depth': 14, 'subsample': 0.6228710088366072, 'colsample_bytree': 0.5208097233791955, 'reg_alpha': 3.07527433979948, 'reg_lambda': 2.603865463210192}. Best is trial 1 with value: 0.477071541779303.


...

✅ Completed Trial 3 - MAPE: 0.3819
🔄 Starting Trial 4 out of 5


[I 2025-01-30 12:30:57,615] Trial 3 finished with value: 0.5377703983136883 and parameters: {'n_estimators': 157, 'learning_rate': 0.04014636574478453, 'max_depth': 18, 'subsample': 0.8693792119211561, 'colsample_bytree': 0.5298794949801255, 'reg_alpha': 7.073489627090692, 'reg_lambda': 7.4992558111335414}. Best is trial 2 with value: 0.38192155099568004.


✅ Completed Trial 4 - MAPE: 0.5378
🔄 Starting Trial 5 out of 5


[W 2025-01-30 12:31:05,843] Trial 4 failed with parameters: {'n_estimators': 387, 'learning_rate': 0.09062129748555271, 'max_depth': 16, 'subsample': 0.5762699469681487, 'colsample_bytree': 0.955998208847619, 'reg_alpha': 3.2919930528970487, 'reg_lambda': 1.9956472774028984} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Leo\AppData\Local\Temp\ipykernel_17116\70811098.py", line 20, in objective
    model.fit(X_train.values, y_train.values)
  File "c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\xgboost\sklearn.py", line 1108, in fit
    self._Booster = train(
  File "c:\Users\Leo\.pyenv\pyenv-win-venv\envs

KeyboardInterrupt: 

In [49]:
def objective(trial, X_train, X_test, y_train, y_test, total_trials):
    """
    Optuna objective function for XGBoost hyperparameter optimization with GPU support.
    
    Parameters:
    trial: Optuna trial object
    X_train, X_test, y_train, y_test: Training and test data
    total_trials: Total number of trials to run
    """
    trial_start = time.time()
    print(f"🔄 Starting Trial {trial.number + 1} out of {total_trials}")
    
    # Check GPU availability
    gpu_available = torch.cuda.is_available()
    device = "gpu" if gpu_available else "cpu"
    tree_method = "hist" if gpu_available else "auto"
    
    # Define hyperparameters with reasonable ranges
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "device": device,
        "tree_method": tree_method
    }
    
    try:
        # Convert data to device-appropriate format
        if device == "gpu":
            X_train_device = xgb.DMatrix(X_train, y_train)
            X_test_device = xgb.DMatrix(X_test)
        else:
            X_train_device = X_train
            X_test_device = X_test
        
        # Train the model with early stopping
        model = xgb.XGBRegressor(
            **param,
            early_stopping_rounds=50,
            eval_metric=['mae', 'mape']
        )
        
        model.fit(
            X_train_device, y_train,
            eval_set=[(X_test_device, y_test)],
            verbose=False
        )
        
        # Make predictions
        preds = model.predict(X_test_device)
        mape = mean_absolute_percentage_error(y_test, preds)
        
        # Calculate additional metrics
        mae = np.mean(np.abs(y_test - preds))
        rmse = np.sqrt(np.mean((y_test - preds) ** 2))
        
        # Log results
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('rmse', rmse)
        trial.set_user_attr('n_estimators_used', model.best_iteration or param['n_estimators'])
        
        trial_time = time.time() - trial_start
        print(f"✅ Trial {trial.number + 1} completed in {trial_time:.1f}s")
        print(f"   MAPE: {mape:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return mape
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        raise optuna.exceptions.TrialPruned()

def run_optimization(X_train, X_test, y_train, y_test, n_trials=5):
    """
    Run the complete optimization process and return the best model.
    """
    study = optuna.create_study(direction="minimize")
    
    try:
        study.optimize(
            lambda trial: objective(trial, X_train, X_test, y_train, y_test, n_trials),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        # Get best parameters and create final model
        best_params = study.best_params
        best_params.update({
            "enable_categorical": True,
            "device": "gpu" if torch.cuda.is_available() else "cpu",
            "tree_method": "hist" if torch.cuda.is_available() else "auto"
        })
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print("\n🏆 Best Trial Results:")
        print(f"MAPE: {study.best_value:.4f}")
        print("Best Parameters:", best_params)
        
        return best_model, study
        
    except KeyboardInterrupt:
        print("\n⚠️ Optimization interrupted by user")
        return None, study


best_model, study = run_optimization(X_train, X_test, y_train, y_test, n_trials=15)

# Make predictions with best model
if best_model is not None:
    predictions = best_model.predict(X_test)
    final_mape = mean_absolute_percentage_error(y_test, predictions)
    print(f"Final MAPE: {final_mape:.4f}")

[I 2025-01-30 12:45:51,078] A new study created in memory with name: no-name-0b7516cc-1c37-44c9-bc62-43a6b4e6c8c0


  0%|          | 0/15 [00:00<?, ?it/s]

🔄 Starting Trial 1 out of 15
✅ Trial 1 completed in 3.0s
   MAPE: 1.2029, MAE: 92.0556, RMSE: 116.8164
[I 2025-01-30 12:45:54,116] Trial 0 finished with value: 1.2028849659894678 and parameters: {'n_estimators': 226, 'learning_rate': 0.0140989437140282, 'max_depth': 7, 'subsample': 0.9358988117079101, 'colsample_bytree': 0.5696372790613986, 'reg_alpha': 0.5184694706783463, 'reg_lambda': 0.1568892955675274, 'min_child_weight': 7, 'gamma': 0.5291305330285995}. Best is trial 0 with value: 1.2028849659894678.
🔄 Starting Trial 2 out of 15
✅ Trial 2 completed in 4.6s
   MAPE: 0.3540, MAE: 53.5593, RMSE: 94.2412
[I 2025-01-30 12:45:58,733] Trial 1 finished with value: 0.3540062660549229 and parameters: {'n_estimators': 312, 'learning_rate': 0.05001841889601309, 'max_depth': 12, 'subsample': 0.5563484558428962, 'colsample_bytree': 0.731307914749396, 'reg_alpha': 0.6842499592232841, 'reg_lambda': 0.7576098080671874, 'min_child_weight': 4, 'gamma': 1.992806812179479}. Best is trial 1 with value:

In [119]:
#Catboost
def objective_cb(trial):
    print(f"🔄 Starting CatBoost Trial {trial.number + 1} out of {20}") 

    param = {
        "iterations": trial.suggest_int("iterations", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-5, 10.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "Poisson"]),
        "task_type": "GPU",  # Enable GPU usage
        "devices": "0"
    }

    # Ensure subsample is only used when bootstrap_type supports it
    if param["bootstrap_type"] in ["Bernoulli", "Poisson"]:
        param["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)

    model = cb.CatBoostRegressor(**param, verbose=0)
    model.fit(X_train.values, y_train.values)

    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)

    print(f"✅ Completed CatBoost Trial {trial.number + 1} - MAPE: {mape:.4f}")
    return mape

# Run Optuna study for CatBoost
study_cb = optuna.create_study(direction="minimize")
study_cb.optimize(objective_cb, n_trials=5)


[I 2025-01-30 10:32:17,429] A new study created in memory with name: no-name-ca9d989e-e7d7-4a3b-ba50-cfc0531a8b02


🔄 Starting CatBoost Trial 1 out of 20


[I 2025-01-30 10:32:19,864] Trial 0 finished with value: 0.5642048639613763 and parameters: {'iterations': 110, 'learning_rate': 0.07761315610356374, 'depth': 10, 'l2_leaf_reg': 1.287032171603146e-05, 'bootstrap_type': 'Bernoulli', 'subsample': 0.993958007682485}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 1 - MAPE: 0.5642
🔄 Starting CatBoost Trial 2 out of 20


[I 2025-01-30 10:32:21,459] Trial 1 finished with value: 1.2054574011116603 and parameters: {'iterations': 183, 'learning_rate': 0.045263313626500004, 'depth': 3, 'l2_leaf_reg': 0.0005504960257199449, 'bootstrap_type': 'Bayesian'}. Best is trial 0 with value: 0.5642048639613763.


...

✅ Completed CatBoost Trial 3 - MAPE: 0.6217
🔄 Starting CatBoost Trial 4 out of 20


[I 2025-01-30 10:32:24,749] Trial 3 finished with value: 1.3690629686356102 and parameters: {'iterations': 135, 'learning_rate': 0.013700592645527324, 'depth': 5, 'l2_leaf_reg': 7.120787883299445e-05, 'bootstrap_type': 'Poisson', 'subsample': 0.7280860114280876}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 4 - MAPE: 1.3691
🔄 Starting CatBoost Trial 5 out of 20


[I 2025-01-30 10:32:26,275] Trial 4 finished with value: 0.6745534215792119 and parameters: {'iterations': 63, 'learning_rate': 0.0703410258316165, 'depth': 8, 'l2_leaf_reg': 0.03814485996533083, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6321310790982253}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 5 - MAPE: 0.6746


In [120]:
#lightGBM
import lightgbm as lgb

def objective_lgb(trial):
    print(f"🔄 Starting Trial {trial.number + 1} out of {20}")

    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),
        "device": "gpu",  # Enable GPU acceleration
        "boosting_type": "gbdt"
    }

    # Train the model
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train.values, y_train.values)

    # Predict & compute MAPE
    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)
    
    print(f"✅ Completed Trial {trial.number + 1} - MAPE: {mape:.4f}")

    return mape

# Run Optuna study for LightGBM
study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective_lgb, n_trials=5)


[I 2025-01-30 10:32:53,263] A new study created in memory with name: no-name-59249260-7cad-411a-87c9-b392b930b2cf


🔄 Starting Trial 1 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.004394 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[I 2025-01-30 10:32:55,016] Trial 0 finished with value: 0.6661336500835574 and parameters: {'n_estimators': 143, 'learning_rate': 0.05421579704191731, 'max_depth': 5, 'subsample': 0.7714021918077434, 'colsample_bytree': 0.7757299648452833, 'reg_alpha': 6.666265064362723, 'reg_lambda': 7.404644607747894}. Best is trial 0 with value: 0.6661336500835574.


✅ Completed Trial 1 - MAPE: 0.6661
🔄 Starting Trial 2 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.003377 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-01-30 10:32:56,419] Trial 1 finished with value: 0.5890301337015383 and parameters: {'n_estimators': 120, 'learning_rate': 0.23917404264091283, 'max_depth': 6, 'subsample': 0.5702624363564109, 'colsample_bytree': 0.5302540189053162, 'reg_alpha': 9.181284904224954, 'reg_lambda': 5.777833963385993}. Best is trial 1 with value: 0.5890301337015383.


...

✅ Completed Trial 3 - MAPE: 0.6306
🔄 Starting Trial 4 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.003020 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-01-30 10:32:59,950] Trial 3 finished with value: 0.6655998943569493 and parameters: {'n_estimators': 100, 'learning_rate': 0.20510095694932834, 'max_depth': 15, 'subsample': 0.6958803783553931, 'colsample_bytree': 0.7942416619544808, 'reg_alpha': 8.023697065904198, 'reg_lambda': 1.432326646475349}. Best is trial 1 with value: 0.5890301337015383.


✅ Completed Trial 4 - MAPE: 0.6656
🔄 Starting Trial 5 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.004067 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-01-30 10:33:01,786] Trial 4 finished with value: 0.6758076443891302 and parameters: {'n_estimators': 135, 'learning_rate': 0.06539363345497198, 'max_depth': 5, 'subsample': 0.7668706791010944, 'colsample_bytree': 0.7899188132034961, 'reg_alpha': 9.583677812457712, 'reg_lambda': 0.7563834387705128}. Best is trial 1 with value: 0.5890301337015383.


✅ Completed Trial 5 - MAPE: 0.6758


In [35]:
X_trial = feature_eng(blind)[1]


In [46]:
holiday_checker = initialize_holiday_detection(blind)
blind['is_holiday'] =blind.apply(holiday_checker, axis=1)

KeyboardInterrupt: 

In [44]:
blind.head(1)

,id,date,country,store,product,is_holiday
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose,False


In [176]:
predictions = final_model.predict(X_trial.values)
submission_df = pd.DataFrame({"predictions": predictions})
submission_df = pd.merge(blind['id'], 
         submission_df,
         left_index=True,
         right_index=True)
submission_file = submission_df.to_csv("submission.csv", index=False)


In [177]:
predictions

array([ 603.00525,  822.55347,  717.7928 , ..., 1864.3871 , 1046.0208 ,
       1267.8745 ], dtype=float32)

In [178]:
submission_df

,id,predictions
0,230130,603.005249
1,230131,822.553467
2,230132,717.792786
3,230133,390.541016
4,230134,443.878021
...,...,...
98545,328675,401.625610
98546,328676,2251.829102
98547,328677,1864.387085
98548,328678,1046.020752


In [170]:
submission_file

In [166]:
predictions

array([ 610.90393,  838.7246 ,  677.8083 , ..., 1953.422  , 1048.2671 ,
       1243.5255 ], dtype=float32)

In [173]:
submission_df

,id,predictions
0,230130,610.903931
1,230131,838.724609
2,230132,677.808289
3,230133,368.214539
4,230134,438.093872
...,...,...
98545,328675,374.361359
98546,328676,2303.772705
98547,328677,1953.421997
98548,328678,1048.267090


In [ ]:
blind_predition_model = model.fit(test)

NameError: name 'model' is not defined